In [74]:
!pip install psycopg2

import psycopg2

username = "sueszli"
password = "password"
host = "localhost"
port = "5432"
database = "postgres"

# test connection
conn = psycopg2.connect(
    user=username,
    password=password,
    host=host,
    port=port,
    database=database
)

In [75]:
# reset database
!chmod +x ./reset.sh && ./reset.sh >> /dev/null

./reset.sh: line 10: return: can only `return' from a function or sourced script
Did not find any relations.
ERROR:  cannot drop the currently open database
ERROR:  current user cannot be dropped


In [116]:
!psql postgres -c "explain (analyze, costs off) $query"

                                        QUERY PLAN                                        
------------------------------------------------------------------------------------------
 Merge Join (actual time=132.082..323.195 rows=2008672 loops=1)
   Merge Cond: ((t.c = s.c) AND (t.a = r.a))
   ->  Sort (actual time=10.965..11.744 rows=10264 loops=1)
         Sort Key: t.c, t.a
         Sort Method: quicksort  Memory: 1394kB
         ->  Seq Scan on t (actual time=0.011..1.700 rows=20000 loops=1)
   ->  Materialize (actual time=121.110..199.709 rows=2216058 loops=1)
         ->  Sort (actual time=121.096..143.440 rows=413972 loops=1)
               Sort Key: s.c, r.a
               Sort Method: external merge  Disk: 8960kB
               ->  Hash Join (actual time=0.989..27.898 rows=413972 loops=1)
                     Hash Cond: (r.b = s.b)
                     ->  Seq Scan on r (actual time=0.022..0.988 rows=20000 loops=1)
                     ->  Hash (actual time=0.947..0.947 rows=20

In [119]:
import os
import json

QUERY = "SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"

def get_json(query: str) -> str:
    output = os.popen(f'psql postgres -c "explain (analyze, verbose, costs, settings, buffers, wal, timing, summary, format json) {query};"').read()
    output = output.splitlines()[2:-2]
    for i in range(len(output)):
        if len(output[i]) > 2:
            output[i] = output[i][:-1]
    output = "\n".join(output)
    output = json.dumps(json.loads(output), indent=2)
    return output

def dump_to_file(output: str, output_path: str="dump.json"):
    with open(output_path, "w") as f:
        f.write(output)

res = get_json(QUERY)
dump_to_file(res)
print("done")


done
